## Imports


In [2]:
from collections import Counter
import os

from nltk import word_tokenize
from nltk.corpus import stopwords

import speech_recognition as sr

from pydub import AudioSegment
from pydub.silence import split_on_silence

## Question 1

- Reads a large audio file that contains speech
- Splits into several chunks of audio files (i.e., "chunk_0.wav", "chunk_1.wav", ...).


In [40]:
f = "given_files/alice_in_wonderlandch01.mp3"

audio_data = AudioSegment.from_file(f)

min_silence_len = 1000
silence_thresh = audio_data.dBFS - 20
keep_silence = 100

chunks = split_on_silence(
    audio_data,
    min_silence_len=min_silence_len,
    silence_thresh=silence_thresh,
    keep_silence=keep_silence,
)

if not os.path.exists("audio_chunks"):
    print("Making audio_chunks directory")
    os.makedirs("audio_chunks")

for i, chunk in enumerate(chunks):
    chunk.export(f"audio_chunks/chunk_{i}.wav", format="wav")

Making audio_chunks directory


## Question 2

Transcribes the audio files and saves the transcribed text into corresponding text files (i.e., "text_0.txt", "text_1.txt", ...)


In [41]:
r = sr.Recognizer()

audio_chunks = os.listdir("audio_chunks")

# Make sure chunks are sorted
sorted_chunks = sorted(
    audio_chunks, key=lambda c: int(c.split("_")[-1].split(".")[0])
)  # sort by file #

if not os.path.exists("audio_texts"):
    print("Making audio_texts directory")
    os.makedirs("audio_texts")

for i, audio_chunk in enumerate(sorted_chunks):
    with sr.AudioFile(f"audio_chunks/{audio_chunk}") as source:
        audio = r.record(source)
        text = r.recognize_google(audio)

    with open(f"audio_texts/text_{i}.txt", "w") as f:
        f.write(text)

Making audio_texts directory


## Question 3

Creates a BoW with their #s of occurrences in ALL the text files.
Make sure to do this by converting all the words into either upper- or lower-case letters (i.e., any words with the same letters should be recognized as the same words regardless of the letter cases).


In [42]:
BoW = Counter()

audio_texts = os.listdir("audio_texts")

for i, audio_text in enumerate(audio_texts):
    with open(f"audio_texts/{audio_text}", "r") as f:
        text = f.read()
        # Make all lower tokens case
        tokens = [word.lower() for word in word_tokenize(text)]

        BoW.update(tokens)

print(BoW)

Counter({'the': 90, 'she': 79, 'to': 73, 'and': 64, 'it': 63, 'a': 54, 'was': 52, 'of': 43, 'i': 35, 'down': 30, 'that': 29, 'her': 26, 'alice': 25, 'in': 24, 'but': 22, 'very': 21, 'you': 19, 'for': 19, 'had': 19, 'on': 17, 'not': 16, 'little': 15, 'be': 14, 'so': 14, 'way': 14, 'as': 14, 'out': 13, 'up': 13, 'this': 13, 'me': 13, 'herself': 13, 'with': 12, 'there': 12, 'at': 12, 'like': 11, 'think': 11, 'all': 11, 'if': 10, 'see': 10, 'no': 10, 'or': 10, 'what': 10, 'when': 10, 'into': 9, 'how': 9, 'could': 9, 'time': 9, 'which': 9, "'s": 9, 'one': 9, 'rabbit': 9, 'about': 8, 'would': 8, 'were': 8, 'by': 8, 'do': 8, 'door': 7, 'found': 7, 'get': 7, 'is': 7, 'nothing': 7, 'much': 6, 'through': 6, 'went': 6, 'before': 6, 'said': 6, 'say': 6, 'going': 6, "'ll": 6, 'they': 6, 'off': 6, 'wonder': 6, 'then': 6, 'my': 5, 'use': 5, 'know': 5, 'things': 5, 'table': 5, 'key': 5, 'too': 5, 'never': 5, 'tried': 5, 'thought': 5, 'suddenly': 5, 'did': 5, 'shall': 5, 'either': 5, "n't": 5, 'looked'

## Question 4

Removes any words in the BoW that:

- are punctuations
- are stopwords including separate abbreviations ["n't","'ll","'s"]
- Appear less than 5 times in all the corpora.


In [43]:
# No need to remove punctuation

stop_words = stopwords.words("english")

additional_words = ["n't", "'ll", "'s"]

# Remove stopwords including additional ones
stop_words = set(stop_words).union(additional_words)
BoW = Counter({word: freq for word, freq in BoW.items() if word not in stop_words})

# Remove words with a frequency of less than 5
BoW = Counter({word: freq for word, freq in BoW.items() if freq >= 5})

# Convert the BoW to a list and sort by frquencies
BoW = sorted(BoW.items(), key=lambda word: word[1], reverse=True)

print(BoW)

[('alice', 25), ('little', 15), ('way', 14), ('like', 11), ('think', 11), ('see', 10), ('could', 9), ('time', 9), ('one', 9), ('rabbit', 9), ('would', 8), ('door', 7), ('found', 7), ('get', 7), ('nothing', 7), ('much', 6), ('went', 6), ('said', 6), ('say', 6), ('going', 6), ('wonder', 6), ('use', 5), ('know', 5), ('things', 5), ('table', 5), ('key', 5), ('never', 5), ('tried', 5), ('thought', 5), ('suddenly', 5), ('shall', 5), ('either', 5)]


## Question 5

Run the Huffman encoding on the words in the BoW.


In [44]:
## Helper code to create the encoding. Same as submitted in the bonus quiz


# This code was taken from "rec_nov20_2023" and
# https://www.programiz.com/dsa/huffman-coding
# It is slightly modified so that it works with python and numpy strings
class TreeNode(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)


# Function to generate the nodes from the Bag of Words
def generate_nodes(BoW):
    BoW_copy = list(BoW)
    while len(BoW_copy) > 1:
        key1, c1 = BoW_copy.pop()
        key2, c2 = BoW_copy.pop()

        node = TreeNode(key1, key2)
        BoW_copy.append((node, c1 + c2))

        BoW_copy = sorted(BoW_copy, key=lambda w: w[1], reverse=True)
    return BoW_copy


# Main function implementing huffman coding
def huffman_code_tree(node, left=True, binString=""):
    # Play nice with numpy
    if type(node) == str:
        return {node: binString}

    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + "0"))
    d.update(huffman_code_tree(r, False, binString + "1"))
    return d

In [45]:
nodes = generate_nodes(BoW)

huffman_code = huffman_code_tree(nodes[0][0])

print("Word       Bits         Freq")
print("----------------------------")
for word, frequency in BoW:
    bits = huffman_code[word]
    print(f"{word:8}   {bits:10}   {frequency:4}")

Word       Bits         Freq
----------------------------
alice      010            25
little     1001           15
way        0111           14
like       0000           11
think      11111          11
see        11001          10
could      10101           9
time       10100           9
one        10111           9
rabbit     10110           9
would      10001           8
door       10000           7
found      01101           7
get        01100           7
nothing    00111           7
much       00110           6
went       00011           6
said       00010           6
say        00101           6
going      00100           6
wonder     111101          6
use        111100          5
know       111001          5
things     111000          5
table      111011          5
key        111010          5
never      110101          5
tried      110100          5
thought    110111          5
suddenly   110110          5
shall      110001          5
either     110000          5


## Questions 6

- Asks for one of the codes to be used to use the corresponding word as a search key.
- Search the corpora and display all the texts that contain the corresponding word.


In [46]:
# Helper functions
def find_by_bits(key):
    target = [w for w in huffman_code if huffman_code[w] == key]

    if target:
        target = target[0]
        print(f"'{target}' is found in the following text:\n")

        audio_texts = os.listdir("audio_texts")

        sorted_texts = sorted(
            audio_texts, key=lambda t: int(t.split("_")[-1].split(".")[0])
        )  # sort by file #

        for audio_text in sorted_texts:
            with open(f"audio_texts/{audio_text}", "r") as f:
                text = f.read()
                # Match lower cases
                if target in text.lower().split():
                    print(f"{audio_text}: {text}")
                    continue  # Skip further search in text

    else:
        print(f"'{key}' does not correspond to any words in the Huffman code")

In [50]:
key = input("Enter the code/bits to select a word: ")

if key.isnumeric():
    find_by_bits(key)
else:
    print("Invalid input. Try again")

'alice' is found in the following text:

text_0.txt: Alice in Wonderland by Lewis Carroll chapter 1 down the rabbit
text_1.txt: Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures of conversation so she was considering in her own mind as well as she could for the hot day made her feel very sleepy and stupid whether the pleasure of making a daisy chain would be worth the trouble of getting up and picking the daisies when suddenly a white rabbit with pink eyes ran close by her there was nothing so very remarkable in that nor did Alice think it's so very much out of the way to hear the rabbit say to itself oh dear oh dear I shall be late when she thought it over afterwards it occurred to her that she ought to have wondered at this but at the time it all seemed quite na